<a href="https://colab.research.google.com/github/ykitaguchi77/FundusPhoto/blob/main/AgeEstimation_optos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AgeEstimation_Optos**

##**Load pretrained weight**

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 必要ライブラリ読み込み
!pip install timm==0.5.4 --q #timmのバージョンが違うとうまくstate dictできないので注意！！！
import random
import timm
import copy
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from timm.scheduler import CosineLRScheduler
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
import os
import glob
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_squared_error, r2_score
import time
import matplotlib.pyplot as plt
%matplotlib inline




# モデル枠組み読み込み
model = timm.create_model(model_name='swin_base_patch4_window12_384', num_classes=1, pretrained=False)

# GPU使用する場合
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

# 学習済みモデル読み込み
model_path = '/content/drive/MyDrive/Deep_learning/Fundus_metabolic/model_20220903.pth'
#model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.load_state_dict(torch.load(model_path))


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 10.5 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


cuda:0


<ipython-input-1-a1c09bba1014>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [2]:
optos_dir = "/content/drive/MyDrive/Deep_learning/FundusPhoto/Optos"
jpg_files = [f for f in os.listdir(optos_dir) if f.endswith('.jpg')]

# 結果を表示
jpg_files


['211034_20241017_114403_Optos_L_64_M.jpg',
 '210907_20240924_113602_Optos_L_52_M.jpg',
 '210906_20240924_113252_Optos_R_52_M.jpg',
 '210810_20240904_155929_Optos_R_46_F.jpg',
 '210801_20240903_111748_Optos_R_50_F.jpg',
 '210800_20240902_162824_Optos_R_50_F.jpg',
 '210799_20240902_152327_Optos_L_75_M.jpg',
 '210797_20240902_134304_Optos_L_78_M.jpg',
 '210781_20241018_105426_Optos_L_57_M.jpg']

In [ ]:
import torch
import timm
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image
import os
import pandas as pd

# 設定
model_path = '/content/drive/MyDrive/Deep_learning/Fundus_metabolic/models/finetuned_model/age.pth'
img_dir = "/content/drive/MyDrive/Deep_learning/FundusPhoto/Optos"

# デバイス設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 画像変換設定
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 画像を読み込み、指定されたセンタークロップ率、正方形クロップ、384x384にリサイズ
def preprocess_image(img_path, crop_percentage=0.0, target_size=384):
    image = Image.open(img_path).convert('RGB')

    # 指定した割合でセンタークロップ
    width, height = image.size
    crop_left = width * crop_percentage
    crop_top = height * crop_percentage
    crop_right = width * (1 - crop_percentage)
    crop_bottom = height * (1 - crop_percentage)
    image = image.crop((crop_left, crop_top, crop_right, crop_bottom))

    # 短辺に合わせて正方形にクロップ
    width, height = image.size
    min_dim = min(width, height)
    left = (width - min_dim) // 2
    top = (height - min_dim) // 2
    right = left + min_dim
    bottom = top + min_dim
    image = image.crop((left, top, right, bottom))

    # 384x384にリサイズ
    image = image.resize((target_size, target_size), Image.LANCZOS)

    return image

# モデルの読み込み
model = timm.create_model('swin_base_patch4_window12_384', num_classes=1, pretrained=False)
model.load_state_dict(torch.load(model_path))
model.eval()
model = model.to(device)

# 結果を格納するリスト
file_names = []
true_ages = []
crop_percentages = [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]
predicted_ages = {p: [] for p in crop_percentages}  # 各クロップ率ごとの推定年齢を保持

# img_dir 内のすべての画像に対して推論を行い、各センタークロップ率の変化を記録
for file_name in os.listdir(img_dir):
    if file_name.endswith('.jpg'):
        img_path = os.path.join(img_dir, file_name)

        # 正解年齢をファイル名から取得
        true_age = int(file_name.split('_')[-2])

        # 各センタークロップ率で前処理と推論を実施
        for crop_percentage in crop_percentages:
            # 画像の前処理
            processed_image = preprocess_image(img_path, crop_percentage=crop_percentage)
            image_tensor = transform(processed_image).unsqueeze(0).to(device)

            # 推論
            with torch.no_grad():
                pred_age = model(image_tensor).item()

            # 結果をリストに追加
            if crop_percentage == 0:
                file_names.append(file_name)
                true_ages.append(true_age)
            predicted_ages[crop_percentage].append(pred_age)

            # 前処理した画像と推論結果の表示
            plt.imshow(processed_image)
            plt.title(f"{file_name} - Crop {int(crop_percentage * 100)}% - Predicted Age: {pred_age:.2f}")
            plt.axis('off')
            plt.show()

# DataFrameに変換して表示
df_results = pd.DataFrame({
    "File Name": file_names,
    "True Age": true_ages,
    "Pred Age (0%)": predicted_ages[0],
    "Pred Age (5%)": predicted_ages[0.05],
    "Pred Age (10%)": predicted_ages[0.1],
    "Pred Age (15%)": predicted_ages[0.15],
    "Pred Age (20%)": predicted_ages[0.2],
    "Pred Age (25%)": predicted_ages[0.25],
    "Pred Age (30%)": predicted_ages[0.3],
    "Pred Age (35%)": predicted_ages[0.35],
    "Pred Age (40%)": predicted_ages[0.4]
})

# DataFrame表示
print(df_results)


In [26]:
# 1. 各予測列と True Age の間の平均絶対誤差（MAE）を計算
pred_columns = [col for col in df_results.columns if 'Pred Age' in col]
mae_list = []
print("各予測列のMAE:")
for col in pred_columns:
    mae = np.mean(np.abs(df_results['True Age'] - df_results[col]))
    mae_list.append(mae)
    print(f"{col}: MAE = {mae:.4f}")

# MAEの統計値を計算
mae_min = min(mae_list)
mae_max = max(mae_list)
mae_median = np.median(mae_list)
mae_mean = np.mean(mae_list)

print("\nMAEの統計:")
print(f"最小値: {mae_min:.4f}")
print(f"最大値: {mae_max:.4f}")
print(f"中央値: {mae_median:.4f}")
print(f"平均値: {mae_mean:.4f}")

# 2. エラーベースの重み付けを計算（MAEの逆数を使用）
weights = [1 / mae for mae in mae_list]
total_weight = sum(weights)
normalized_weights = [w / total_weight for w in weights]

# 重みを表示
print("\n各予測列の重み（正規化済み）:")
for col, weight in zip(pred_columns, normalized_weights):
    print(f"{col}: Weight = {weight:.4f}")

# 3. 加重平均を計算してアンサンブル予測を行う
df_results['Ensemble Prediction'] = 0
for col, weight in zip(pred_columns, normalized_weights):
    df_results['Ensemble Prediction'] += df_results[col] * weight

# 4. アンサンブル予測の性能評価
ensemble_mae = np.mean(np.abs(df_results['True Age'] - df_results['Ensemble Prediction']))
print(f"\nアンサンブル予測のMAE: {ensemble_mae:.4f}")

# 5. 結果の比較
print("\n結果の比較:")
display_columns = ['File Name', 'True Age', 'Ensemble Prediction'] + pred_columns
print(df_results[display_columns])

各予測列のMAE:
Pred Age (0%): MAE = 9.4963
Pred Age (5%): MAE = 10.9371
Pred Age (10%): MAE = 10.8579
Pred Age (15%): MAE = 8.8093
Pred Age (20%): MAE = 8.3415
Pred Age (25%): MAE = 7.4064
Pred Age (30%): MAE = 6.6796
Pred Age (35%): MAE = 5.8097
Pred Age (40%): MAE = 8.4999

MAEの統計:
最小値: 5.8097
最大値: 10.9371
中央値: 8.4999
平均値: 8.5375

各予測列の重み（正規化済み）:
Pred Age (0%): Weight = 0.0960
Pred Age (5%): Weight = 0.0834
Pred Age (10%): Weight = 0.0840
Pred Age (15%): Weight = 0.1035
Pred Age (20%): Weight = 0.1093
Pred Age (25%): Weight = 0.1231
Pred Age (30%): Weight = 0.1365
Pred Age (35%): Weight = 0.1569
Pred Age (40%): Weight = 0.1073

アンサンブル予測のMAE: 7.8508

結果の比較:
                                 File Name  True Age  Ensemble Prediction  \
0  211034_20241017_114403_Optos_L_64_M.jpg        64            61.308749   
1  210907_20240924_113602_Optos_L_52_M.jpg        52            52.717928   
2  210906_20240924_113252_Optos_R_52_M.jpg        52            59.493920   
3  210810_20240904_155929_Opto

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# サブプロットのグリッドを作成
fig, axs = plt.subplots(3, 3, figsize=(15, 15))
fig.suptitle('True Age vs Predicted Age by Augmentation Degree', fontsize=16, y=1.02)

# フラットなaxsリストを作成
axs_flat = axs.flatten()

# 各拡大率についてプロット
for idx, crop_percentage in enumerate(crop_percentages):
    ax = axs_flat[idx]

    # データ準備
    true_age = df_results['True Age']
    pred_age = df_results[f'Pred Age ({int(crop_percentage*100)}%)']

    # 散布図
    sns.scatterplot(x=true_age, y=pred_age, ax=ax, color=palette[2], alpha=0.6)

    # 完全一致線（y=x）を追加
    min_age = min(min(true_age), min(pred_age))
    max_age = max(max(true_age), max(pred_age))
    ax.plot([min_age, max_age], [min_age, max_age], '--', color='gray', alpha=0.5)

    # R2スコアを計算
    r2 = np.corrcoef(true_age, pred_age)[0, 1]**2

    # MAEを計算
    mae = np.mean(np.abs(true_age - pred_age))

    # タイトルと軸ラベル
    ax.set_title(f'Augmentation {int(crop_percentage*100)}%\nR² = {r2:.3f}, MAE = {mae:.2f}')
    ax.set_xlabel('True Age')
    ax.set_ylabel('Predicted Age')

    # 軸の範囲を統一
    ax.set_xlim(min_age-5, max_age+5)
    ax.set_ylim(min_age-5, max_age+5)

    # グリッド追加
    ax.grid(True, linestyle='--', alpha=0.3)

# レイアウトの調整
plt.tight_layout()
plt.show()

# すべての拡大率での相関係数とMAEを表示
print("\n詳細な統計結果:")
for crop_percentage in crop_percentages:
    true_age = df_results['True Age']
    pred_age = df_results[f'Pred Age ({int(crop_percentage*100)}%)']

    r2 = np.corrcoef(true_age, pred_age)[0, 1]**2
    mae = np.mean(np.abs(true_age - pred_age))

    print(f"\n拡大率 {int(crop_percentage*100)}%:")
    print(f"R² score: {r2:.4f}")
    print(f"MAE: {mae:.4f}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

# 1. データの準備
results = []
for col in pred_columns:
    errors = np.abs(df_results['True Age'] - df_results[col])
    mae = np.mean(errors)
    se = stats.sem(errors)
    ci = se * stats.t.ppf((1 + 0.95) / 2, len(errors)-1)

    # パーセンテージを抽出 (例: "Pred Age (5%)" から 5 を取得)
    magnification = int(col.split('(')[1].split('%')[0])

    results.append({
        'magnification': magnification,
        'mae': mae,
        'ci': ci
    })

# 拡大率でソート
results = sorted(results, key=lambda x: x['magnification'])

# 2. グラフのスタイル設定
sns.set_style("whitegrid")
plt.figure(figsize=(12, 7))

# カラーパレットの設定
palette = sns.color_palette("flare")

# データの準備
magnifications = [r['magnification'] for r in results]
maes = [r['mae'] for r in results]
cis = [r['ci'] for r in results]

# メインの折れ線プロット
ax = sns.lineplot(x=magnifications, y=maes, marker='o', markersize=10,
                 color=palette[2], linewidth=3, zorder=3)

# 信頼区間の表示
plt.fill_between(magnifications,
                 [mae - ci for mae, ci in zip(maes, cis)],
                 [mae + ci for mae, ci in zip(maes, cis)],
                 color=palette[2], alpha=0.2)

# グラフの装飾
plt.title('Mean Absolute Error by Degree of Augmentation',
          fontsize=16, pad=20, fontweight='bold')
plt.xlabel('Augmentation Degree (%)', fontsize=12, labelpad=10)
plt.ylabel('Mean Absolute Error (years)', fontsize=12, labelpad=10)

# 数値ラベルの表示
for i, (mag, mae, ci) in enumerate(zip(magnifications, maes, cis)):
    ax.annotate(f'MAE: {mae:.2f}\n±{ci:.2f}',
                xy=(mag, mae),
                xytext=(0, 10),
                textcoords='offset points',
                ha='center',
                va='bottom',
                bbox=dict(boxstyle='round,pad=0.5',
                         fc='white',
                         ec=palette[2],
                         alpha=0.9),
                zorder=4)

# グラフ全体のスタイル調整
sns.despine(left=True, bottom=True)
plt.grid(True, linestyle='--', alpha=0.7)

# 軸のスタイル調整
ax.tick_params(axis='both', which='major', labelsize=10)

# x軸の目盛りを設定
plt.xticks(magnifications)

# 余白の調整
plt.tight_layout()

# グラフの表示
plt.show()

# 3. 詳細な統計結果の出力
print("\n詳細な統計結果:")
for result in results:
    print(f"\n拡大率 {result['magnification']}%:")
    print(f"MAE: {result['mae']:.4f}")
    print(f"95%信頼区間: ±{result['ci']:.4f}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# サブプロットのグリッドを作成
fig, axs = plt.subplots(3, 3, figsize=(15, 15))
fig.suptitle('True Age vs Predicted Age by Augmentation Degree', fontsize=16, y=1.02)

# フラットなaxsリストを作成
axs_flat = axs.flatten()

# 各拡大率についてプロット
for idx, crop_percentage in enumerate(crop_percentages):
    ax = axs_flat[idx]

    # データ準備
    true_age = df_results['True Age']
    pred_age = df_results[f'Pred Age ({int(crop_percentage*100)}%)']

    # 散布図
    sns.scatterplot(x=true_age, y=pred_age, ax=ax, color=palette[2], alpha=0.6)

    # 完全一致線（y=x）を追加
    min_age = min(min(true_age), min(pred_age))
    max_age = max(max(true_age), max(pred_age))
    ax.plot([min_age, max_age], [min_age, max_age], '--', color='gray', alpha=0.5)

    # R2スコアを計算
    r2 = np.corrcoef(true_age, pred_age)[0, 1]**2

    # MAEを計算
    mae = np.mean(np.abs(true_age - pred_age))

    # タイトルと軸ラベル
    ax.set_title(f'Augmentation {int(crop_percentage*100)}%\nR² = {r2:.3f}, MAE = {mae:.2f}')
    ax.set_xlabel('True Age')
    ax.set_ylabel('Predicted Age')

    # 軸の範囲を統一
    ax.set_xlim(min_age-5, max_age+5)
    ax.set_ylim(min_age-5, max_age+5)

    # グリッド追加
    ax.grid(True, linestyle='--', alpha=0.3)

# レイアウトの調整
plt.tight_layout()
plt.show()

# すべての拡大率での相関係数とMAEを表示
print("\n詳細な統計結果:")
for crop_percentage in crop_percentages:
    true_age = df_results['True Age']
    pred_age = df_results[f'Pred Age ({int(crop_percentage*100)}%)']

    r2 = np.corrcoef(true_age, pred_age)[0, 1]**2
    mae = np.mean(np.abs(true_age - pred_age))

    print(f"\n拡大率 {int(crop_percentage*100)}%:")
    print(f"R² score: {r2:.4f}")
    print(f"MAE: {mae:.4f}")